Celem tego ćwiczenia jest stworzenie prostego modelu, który będzie w stanie rozróżniać normalne wiadomości (tzw. `ham`) od wiadomośći typu `spam`. Do przygodowania danych, zostanie wykorzystana technika z `Bag of Words`, która zostanie zaimplementowana bez użycia biblioteki `sklearn` w celu zrozumienia jej dokładniejszego działania.

**W tym ćwiczeniu będzie dużo mniej szczegółowych opisów :)**

## Importy

In [182]:
import pandas as pd

## Wartości stałe

In [183]:
RANDOM_SEED = 44

## Wczytanie danych

Dane pochodzą z https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection


In [184]:
df = pd.read_table("https://www.dropbox.com/s/0jb18jj8ve8xh2c/SMSSpamCollection?dl=1",
                   sep='\t', header=None, names=['label', 'sms_message'])

df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Przegląd danych

### - Wyświetl z iloma wierszami mamy do czynienia

In [185]:
number_of_samples = df.shape[0]
print("Ilośc próbek: {}".format(number_of_samples))

Ilośc próbek: 5572


### - Wyświetl unikalne wartości kolumny `label`

In [186]:
unique_values = df["label"].unique()
print("Klasy, którch model będzie uczony: {}".format(unique_values))

Klasy, którch model będzie uczony: ['ham' 'spam']


### - Wyświetl ilość próbek należacych odpowiednio do każdej klasy z osobna

In [187]:
df["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

### - Wyświetl 3 wiadomości typu `ham` oraz 3 wiadomości typu `spam`.

In [188]:
df_ham = df.loc[df["label"] == "ham"]
df_spam = df.loc[df["label"] == "spam"]

In [189]:
print(df_ham["sms_message"].iloc[17])

Just forced myself to eat a slice. I'm really not hungry tho. This sucks. Mark is getting worried. He knows I'm sick when I turn down pizza. Lol


In [190]:
print(df_ham["sms_message"].iloc[1589])

Its on in engalnd! But telly has decided it won't let me watch it and mia and elliot were kissing! Damn it!


In [191]:
print(df_ham["sms_message"].iloc[3242])

Hows the street where the end of library walk is?


In [192]:
print(df_spam["sms_message"].iloc[523])

Free Msg: get Gnarls Barkleys "Crazy" ringtone TOTALLY FREE just reply GO to this message right now!


In [193]:
print(df_spam["sms_message"].iloc[234])

For ur chance to win a £250 cash every wk TXT: ACTION to 80608. T's&C's www.movietrivia.tv custcare 08712405022, 1x150p/wk


In [194]:
print(df_spam["sms_message"].iloc[485])

Get the official ENGLAND poly ringtone or colour flag on yer mobile for tonights game! Text TONE or FLAG to 84199. Optout txt ENG STOP Box39822 W111WX £1.50


### - Zastanów się:
- co jest wartością docelową (tym co model będzie musiał przewidywać)
- która cecha będzie użyta do uczenia modelu
- czy ilośc próbek przypadających na każdą klasę jest równa
- jakiej metryki warto użyć do walidacji modelu

## Przygotowanie danych

### - Kolumna `label` powinna zawierać wartośći binarne 0 lub 1
Użyj obiektu `LabelEncoder` biblioteki `sklearn` albo poeksperymentuj z funkcją `.map` obiektu `Dataframe`.
- link do dokumentacji: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
- link do dokumentacji: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html

In [195]:
df['label'] = df.label.map({'ham': 0, 'spam': 1})

### - Implementacja `Bag of Words` bez użycia `sklearn`

Mając dany następujące wiadomości:

In [196]:
data = ["Hello!, How are you?",
        "Work from home is great.",
        "Do you like ice cream?",
        "I love to sleep at home.",
        "Hello, can I call you now?"]

#### 1. Zamień wszystkie słowa w każdym ze stringów na `lowercase`
Możesz tego dokonać poprzez wywołanie metody `.lower()` na obiekcie string. Następnie zapisz rezultat w liście `data_lowercase`.

In [197]:
data_lowercase = list()

In [198]:
data_lowercase.extend([s.lower() for s in data])

In [199]:
print(data_lowercase)

['hello!, how are you?', 'work from home is great.', 'do you like ice cream?', 'i love to sleep at home.', 'hello, can i call you now?']


#### 2. Usuń wszystkie znaki specjalnie
Stwórz liste punktuacji, których chcesz się pozbyć. 

    punctuations = [".", ","]
    
Następnie przeiteruj po każdym znaku w każdym stringu i omiń te, które należą do tej listy.

In [200]:
punctuations = [".", ",", "!", "?", "-", "_", "&", "$", "£"]    

In [201]:
data_no_punctuations = list()

In [202]:
for string in data_lowercase:
    string_cleaned = ""
    
    for character in string:
        if character not in punctuations:
            string_cleaned += character
    
    data_no_punctuations.append(string_cleaned)

In [203]:
print(data_no_punctuations)

['hello how are you', 'work from home is great', 'do you like ice cream', 'i love to sleep at home', 'hello can i call you now']


#### 3. Obuduj napisany kod w funkcję `clean(string)`.
Funkcja clean powinna zamieniać wszystkie słowa na `lowercase` i usuwać znaki specjalne ze stringa.


In [204]:
def clean(string):
    string_lowercase = string.lower()
    
    punctuations = [".", ",", "!", "?", "-", "_", "&", "$", "£"]    
    
    string_cleaned = ""
    for character in string_lowercase:
        if character not in punctuations:
            string_cleaned += character
    
    return string_cleaned

In [205]:
assert clean("Hello!, How are you?") == "hello how are you"

#### 4. Rozdziel wszystkie słowa i umieść je w liście
Aby tego dokonać użyj metody `split()` na każdym stringu. Docelowo rozdzieli on słowa według znaku `" "` i zapisze w liście, którą następnie trzeba będzie dołączyc do listy `data_splited`.

In [206]:
data_splited = list()

In [207]:
for string in data_no_punctuations:
    words = string.split()
    data_splited.extend(words)

In [208]:
print(data_splited)

['hello', 'how', 'are', 'you', 'work', 'from', 'home', 'is', 'great', 'do', 'you', 'like', 'ice', 'cream', 'i', 'love', 'to', 'sleep', 'at', 'home', 'hello', 'can', 'i', 'call', 'you', 'now']


#### 5. Zlicz częstość występowania każdego słowa w danych
Jesteśmy w stanie tego dokonać dzięki obiektowi `Counter`:
- link do dokumentacji: https://docs.python.org/2/library/collections.html

Api to użycia tego obiektu jest następujące:
    
    counter = Counter(lista_danych)

In [209]:
from collections import Counter

counter = Counter(data_splited)

In [210]:
print(counter)

Counter({'you': 3, 'hello': 2, 'home': 2, 'i': 2, 'how': 1, 'are': 1, 'work': 1, 'from': 1, 'is': 1, 'great': 1, 'do': 1, 'like': 1, 'ice': 1, 'cream': 1, 'love': 1, 'to': 1, 'sleep': 1, 'at': 1, 'can': 1, 'call': 1, 'now': 1})


Następnie można odwołać się do częstości występowania każdego słowa jak do słownika:

In [211]:
counter["home"]

2

#### 6. Stwórz słownik według wzorca `id`:`słowo`
`id` powinny być przydzielone malejąco, według częstośći wsystępowania słowa (najczęsciej występujące słowo `id=0`, kolejne `id=1` itd.)

In [212]:
id_to_word = dict()

sorted_word_freq_list = sorted(list(counter.items()), key = lambda x: x[1], reverse=True) 
for i, (word, frequency) in enumerate(sorted_word_freq_list):
    id_to_word[i] = word

In [213]:
print(id_to_word)

{0: 'you', 1: 'hello', 2: 'home', 3: 'i', 4: 'how', 5: 'are', 6: 'work', 7: 'from', 8: 'is', 9: 'great', 10: 'do', 11: 'like', 12: 'ice', 13: 'cream', 14: 'love', 15: 'to', 16: 'sleep', 17: 'at', 18: 'can', 19: 'call', 20: 'now'}


#### 7. Obuduj napisany kod w funkcję `create_dictionary(list_of_strings)`:
Funkcja:
    
    def create_dictionary(list_of_strings):
        id_to_word = dict()
        
        // miejsce na kod
        
        return id_to_word, word_to_id
        
Powinna przyjmować liste stringów i zwracać słownik `id_to_word` oraz `word_to_id`. Celem stworzenia `id_to_word` jest przypisanie odpowiednich indeksów do słów na bazie ich frekfencji. Słownik `word_to_id` to odwrotność `id_to_word` i jest wygodniejszy do korzystania w momencie tokenizacji.

In [214]:
def create_dictionary(list_of_strings):
    id_to_word = dict()
    word_to_id = dict()

    data_splited = []
    for string in list_of_strings:
        words = string.split()
        data_splited.extend(words)
        
    counter = Counter(data_splited)
    sorted_word_freq_list = sorted(list(counter.items()), key = lambda x: x[1], reverse=True) 
    for i, (word, frequency) in enumerate(sorted_word_freq_list):
        id_to_word[i] = word    
    
    word_to_id = {w: i for (i, w) in id_to_word.items()}
        
    return id_to_word, word_to_id

In [215]:
# Czyszczenie danych
cleaned_data = [clean(s) for s in data]

# Tworzenie słownika
test_id_to_word, _ = create_dictionary(cleaned_data)

assert test_id_to_word == id_to_word

#### 8. Użyj słownika do zamienienia każdego zdania na wektor
Ilość słów w słowniku jest równa wielkości wektora. Każde słowo ma przypisany do siebie indeks równoważny jego pozycji w wektorze. Każda pozycja w wektorze powina zawierać liczbę ile dane słowo, przypisane do tej pozycji, występuje w stringu.

Przykładowo mając dany słownik `id_to_word`:

    {
        0: "the",
        1: "dog",
        2: "over",
        3: "jump",
        4: "sheep"
    }
  
Zdanie:
    
    the dog jumped over the fox
    
Powinno zostać ztokenizowane jako:
    
    [2, 1, 1, 0, 0]

In [216]:
example_string = "the dog jumped over the fox"

id_to_word_example = {
    0: "the",
    1: "dog",
    2: "over",
    3: "jump",
    4: "sheep"
}

word_to_id_example = {w: i for i, w in id_to_word_example.items()}

print(word_to_id_example)

{'the': 0, 'dog': 1, 'over': 2, 'jump': 3, 'sheep': 4}


In [217]:
tokenized_string = [0] * len(example_dictionary)
print("Before: {}".format(tokenized_string))

for word in example_string.split():
    if word in word_to_id_example:
        tokenized_string[word_to_id_example[word]] += 1
        
print("After:  {}".format(tokenized_string))

Before: [0, 0, 0, 0, 0]
After:  [2, 1, 1, 0, 0]


In [218]:
assert tokenized_string == [2, 1, 1, 0, 0]

#### 8. Obuduj kod w funkcję `tokenize(string, word_dict)`
Funkcja powinna przyjmować pojedyńczy string i na bazie przesłanego słownika zamieniać go w odpowiedni wektor.

In [219]:
def tokenize(string, word_dict):
    tokenized_string = [0] * len(word_dict)
    
    for word in string.split():
        if word in word_dict:
            tokenized_string[word_dict[word]] += 1
            
    return tokenized_string

In [220]:
assert tokenize(example_string, word_to_id_example) == [2, 1, 1, 0, 0]

## Podział danych na zbiór testowy/treningowy
- link: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Zaimportuj funkcję `train_test_split` i użyj jej do podzielenia danych na zbiór treningowy/testowy w proporcji `0.8 (train) do 0.2 (test)`. Nie zapomnij ustawić parametru `random_state` na `RANDOM_SEED` aby odtworzyć wynik z notebooka zawierającego odpowiedź.

In [221]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["sms_message"], df["label"], 
                                                    test_size=0.2, random_state=44)

## Tokenizacja
Pora użyć funkcji, które zostały wcześniej:

- wyczyść każdy string w zbiorach `X_train` oraz `X_test` przy użyciu funkcji `clean`,
- zbuduj słownik na bazie zbioru `X_train` (`X_test` symuluje produkcje, więc mogą się tam znaleźć słowa, których model nigdy nie widział) przy użyciu funkcji `create_dictionary`,
- dokonaj tokenizacji stringów ze zbiorów `X_train` oraz `X_test` przy pomocy funkcji `tokenize`.

In [222]:
X_train = [clean(s) for s in X_train]
X_test = [clean(s) for s in X_test]

id_to_word, word_to_id = create_dictionary(X_train)

X_train_tokenized = [tokenize(s, word_to_id) for s in X_train]
X_test_tokenized = [tokenize(s, word_to_id) for s in X_test]

Zwróć uwagę jak wygląda Twój słównik. Jeżeli występują tam znaki specjalnie "przyklejone" do zwykłych słów, to może warto wrócić się do funkcji `clean` i ją poprawić :)

Może usunięcie cyfr to też dobry pomysł?

In [223]:
id_to_word

{0: 'i',
 1: 'to',
 2: 'you',
 3: 'a',
 4: 'the',
 5: 'u',
 6: 'and',
 7: 'in',
 8: 'is',
 9: 'me',
 10: 'my',
 11: 'for',
 12: 'your',
 13: 'it',
 14: 'of',
 15: 'have',
 16: 'call',
 17: 'on',
 18: 'that',
 19: 'are',
 20: '2',
 21: 'now',
 22: 'so',
 23: 'but',
 24: 'not',
 25: 'or',
 26: 'at',
 27: 'do',
 28: 'can',
 29: 'will',
 30: 'be',
 31: 'ur',
 32: 'if',
 33: 'get',
 34: "i'm",
 35: 'with',
 36: 'we',
 37: 'just',
 38: 'this',
 39: 'up',
 40: 'no',
 41: 'when',
 42: 'go',
 43: 'lt;#gt;',
 44: '4',
 45: 'all',
 46: 'from',
 47: 'ok',
 48: 'out',
 49: 'what',
 50: 'free',
 51: 'like',
 52: 'know',
 53: 'how',
 54: 'good',
 55: 'am',
 56: 'then',
 57: 'got',
 58: 'come',
 59: 'was',
 60: 'its',
 61: 'only',
 62: 'there',
 63: 'time',
 64: 'love',
 65: 'he',
 66: 'day',
 67: 'want',
 68: 'as',
 69: 'send',
 70: 'by',
 71: "i'll",
 72: 'going',
 73: 'text',
 74: 'ü',
 75: 'home',
 76: 'one',
 77: 'lor',
 78: 'need',
 79: 'about',
 80: 'r',
 81: 'back',
 82: 'sorry',
 83: 'txt',
 

## Trenowanie modelu

- link: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Zaimportuj model `RandomForestClassifier`. Użyj `.fit` na ztokenizowanym zbiorze treningowym. Użyj `predict` ztokenizowanym zbiorze treningowym oraz testowym. Zapisz wyniki w osobnych zmiennych.

Nie zapomnij ustawić parametru `random_state`.

In [224]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=RANDOM_SEED)
model.fit(X_train_tokenized, y_train)

train_pred = model.predict(X_train_tokenized)
test_pred = model.predict(X_test_tokenized)

## Sprawdzanie jakości modelu

Użyj dowolnej metryki, alby sprawdzić jakość modelu. Wyświetl wynik dla danych treningowych i testowych. 


Zwróć uwagę na rozkład klas. Jest przewaga normalnych wiadomości (ham) nad wiadomościami typu spam. Oznacza to, że accuracy_score nie będzie najlepszą metryką do sprawdzania jakości modelu. Spróbuj poeksperymentować z metrykami takimi jak `f1_score`, `fbeta_score`, `precision`, `recall` a może `auc`? 

In [225]:
from sklearn.metrics import f1_score

train_score = f1_score(train_pred, y_train)
print("Train score: {}".format(train_score))

test_score = f1_score(test_pred, y_test)
print("Test score: {}".format(test_score))

Train score: 0.9921259842519685
Test score: 0.8712871287128713


## Słowa końcowe

Celem tego ćwiczenia było głównie pokazanie, w jaki sposób działa technika Bag of Words i próba zaimplementowania jej samodzielnie. 

Nie nastawiony model `RandomForestClassifier` zwraca wynik blisku 90% poprawności a jednocześnie odrobinę overfittuje. Jeżeli zdecydujesz się na przybliżenie obu wyników (train/test) to pamiętaj, że musisz dodatkowo wydzielić zbiór `validation`, na którym będziesz eksperymentować z parametrami modelu zanim ostatecznie zdecydujesz sie sprawdzić na zbiorze `test`.

Ponieważ ilość klas jest bardzo niezbalansowana musisz upewnić się, że każda klasa została proporcjonalnie podzielona - jeżeli zbiór treningowy to 0.6, walidacyjny to 0.2 i treningowy to 0.2, to w każdym z tych zbiorów powinna się znaleść taka sama proporcja każdej klasy. 

Zalecane jest też stosowanie `cross_validation`. W takich sytuacjach funkcja `StratifiedKFold` jest bardzo pomocna:
- link: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html


Czasami zamiast nastawiać model, warto jest spróbować innego algorytmu. Przykładowo NaiveBayes bardzo dobrze sobie radzi z tym problemem:
- link: http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [226]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

model = MultinomialNB()
model.fit(X_train_tokenized, y_train)

train_pred = model.predict(X_train_tokenized)
test_pred = model.predict(X_test_tokenized)

train_score = f1_score(train_pred, y_train)
print("Train score: {}".format(train_score))

test_score = f1_score(test_pred, y_test)
print("Test score: {}".format(test_score))

Train score: 0.9702797202797203
Test score: 0.9192546583850932
